In [5]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [6]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [9]:
import os
os.environ["OPENAI_API_KEY"] = "your key"
import openai
openai.api_key = "your key"

In [13]:
from sqlalchemy import create_engine, Column, Table, String, MetaData
metadata_obj = MetaData()
engine = create_engine('sqlite:///cats.db', future=True)

cat_breeds_table = Table(
    "cat_breeds",
    metadata_obj,
    Column("name", String(16), primary_key=True),
    Column("origin", String(16), nullable=False),
    Column("temperament", String(16), nullable=False),
    Column("size", String(16), nullable=False),
    Column("social_behavior", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [14]:
from sqlalchemy import insert

breeds_data = [
    {
        'name': 'Savannah',
        'origin': 'USA',
        'temperament': 'high energy',
        'size': 'Large',
        'social_behavior': 'sociable'
    },
    {
        'name': 'Ragdoll',
        'origin': 'USA',
        'temperament': 'gentle',
        'size': 'Large',
        'social_behavior': 'sociable'
    },
    {
        'name': 'Maine Coon',
        'origin': 'USA (Maine)',
        'temperament': 'friendly',
        'size': 'Very Large',
        'social_behavior': 'sociable'
    }
]
for breed in breeds_data:
    stmt = insert(cat_breeds_table).values(**breed)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [15]:
with engine.connect() as connection:
    result = connection.exec_driver_sql("SELECT * FROM cat_breeds")
    print(result.fetchall())

[('Savannah', 'USA', 'high energy', 'Large', 'sociable'), ('Ragdoll', 'USA', 'gentle', 'Large', 'sociable'), ('Maine Coon', 'USA (Maine)', 'friendly', 'Very Large', 'sociable')]


In [18]:
from llama_index.core import SQLDatabase
sql_database = SQLDatabase(engine, include_tables=["cat_breeds"])

In [20]:
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4", temperature=0, verbose=True)
service_context = ServiceContext.from_defaults(llm=llm)


C:\Users\Boris\AppData\Local\Temp\ipykernel_31212\786041.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)


In [27]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine
query_engine = NLSQLTableQueryEngine(
    sql_database,
    context_query_kwargs={"cat_breeds": (
        "The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. \n"
        "Do not attempt to run a query if the column is not among available columns.\n"
        "Do not use non-existant columns in a WHERE part of the query. \n"
    )}
)

In [28]:
openai.log = "debug"

In [29]:
response = query_engine.query("What kind of a temperament does a Ragdoll cat have?")

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'cat_breeds' has columns: name (VARCHAR(16)), origin (VARCHAR(16)), temperament (VARCHAR(16)), size (VARCHAR(16)), social_behavior (VARCHAR(16)), and foreign keys: . The table description is: The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. 
Do not attempt to run a query if the column is not among available columns.
Do not use non-existant columns in a WHERE part of the query. 

> Table desc str: Table 'cat_breeds' has columns: name (VARCHAR(16)), origin (VARCHAR(16)), temperament (VARCHAR(16)), size (VARCHAR(16)), social_behavior (VARCHAR(16)), and foreign keys: . The table description is: The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. 
Do not attempt to run a query if the column is not among available columns.
Do not use non-existant columns in a WHERE part of 

In [30]:
print(response)

Ragdoll cats are known for their gentle temperament. They are typically calm, affectionate, and enjoy being around people.


In [31]:
response.metadata

{'a0cf9c72-a220-4b37-9c2d-18cdc5f950b3': {'sql_query': "SELECT temperament\nFROM cat_breeds\nWHERE name = 'Ragdoll';",
  'result': [('gentle',)],
  'col_keys': ['temperament']},
 'sql_query': "SELECT temperament\nFROM cat_breeds\nWHERE name = 'Ragdoll';",
 'result': [('gentle',)],
 'col_keys': ['temperament']}